# **Prize Collecting Travelling Salesman Problem (PCTSP)**

In [4]:
from random import shuffle
import numpy as np
import random
import re
import copy
import sys

In [5]:
"""
pctsp module - Implements Pctsp, a class that describes an instance of the problem..

"""

class Pctsp(object):
    """
    Attributes:
       c (:obj:`list` of :obj:`list`): Costs from i to j
       p (:obj:`list` of :obj:`int`): Prize for visiting each city i
       gama (:obj:`list` of :obj:`int`): Penalty for not visiting each city i
    """
    def __init__(self, sigma=0.5):
        self.prize = []
        self.penal = []
        self.cost = []
        self.prize_min = 0
        self.sigma = sigma
        self.type = ''
        self.file_name = ''

    def load(self, file_name):
        self.file_name = file_name

        f = open(file_name,'r')
        for i,line in enumerate(f):
            if i is 5: break
            if i is 1: self.prize = np.fromstring(line, dtype=int, sep=' ')
            if i is 4: self.penal = np.fromstring(line, dtype=int, sep=' ')

        f.close()

        self.cost = np.loadtxt(file_name, dtype=int, skiprows=7)
        self.setup()

    def setup(self):
        self.prize_min = np.sum(self.prize)*self.sigma
        self.setup_type()

    def setup_type(self):
        m = re.search(r'problem_(\d+)_((\d+)_(\d+)_(\d+))', self.file_name)

        params = m.group(2)

        if params ==  '100_100_1000':
            self.type = 'A'
        elif params == '100_1000_10000':
            self.type = 'B'
        elif params == '100_100_10000':
            self.type = 'C'

In [6]:
"""
solution module - Implements Solution, a class that describes a solution for the problem.

"""

def solution_random(pctsp, size):
    s = Solution(pctsp)
    length = len(pctsp.prize)

    if size: s.size = size

    i = 0
    num_solutions = 30

    while i < num_solutions or not s.is_valid():
        r = Solution(pctsp)
        if size: r.size = size
        cities = list(range(1, length, 1))
        shuffle(cities) # Shuffle in place
        r.route = [0] + cities # The city 0 is always the first

        if r.quality < s.quality and r.is_valid():
            s = r

        i += 1

    return s

class Solution(object):
    """
    Attributes:
       route (:obj:`list` of :obj:`int`): The list of cities in the visiting order
       size (:obj:`int`): The quantity of the first cities to be considered in the route list
       quality (:obj:`int`): The quality of the solution
    """

    def __init__(self, pctsp, size=None):
        self._route = []
        
        if size:
            self.size = size
        else:
            self.size = len(pctsp.prize) # Default size value is the total of cities
        
        self.quality = sys.maxsize
        self.pctsp = pctsp
        self.prize = 0

    """
    Computes the quality of the solution.
    """
    def compute(self):
        self.prize = 0
        self.quality = 0

        for i,city in enumerate(self._route):
            if i < self.size:
                self.prize += self.pctsp.prize[city]
                if i > 0:
                    previousCity = self._route[i - 1]
                    self.quality += self.pctsp.cost[previousCity][city]
                if i + 1 == self.size:
                    self.quality += self.pctsp.cost[city][0]
            else:
                self.quality += self.pctsp.penal[city]

    def copy(self):
        cp = copy.copy(self)
        cp._route = list(self._route)

        return cp
    
    def swap(self, i, j):
        city_i = self._route[i]
        city_i_prev = self._route[i-1]
        city_i_next = self._route[(i+1) % self.size]
        
        city_j = self._route[j]

        self.quality = (self.quality
                - self.pctsp.cost[city_i_prev][city_i] - self.pctsp.cost[city_i][city_i_next]
                + self.pctsp.cost[city_i_prev][city_j] + self.pctsp.cost[city_j][city_i_next]
                - self.pctsp.penal[city_j] + self.pctsp.penal[city_i])
        self.prize = self.prize - self.pctsp.prize[city_i] + self.pctsp.prize[city_j]

        self._route[j], self._route[i] = self._route[i], self._route[j]

    def is_valid(self):
        return self.prize >= self.pctsp.prize_min

    def add_city(self):
        city_l = self._route[self.size - 1]
        city_add = self._route[self.size]
        
        self.quality = (self.quality
            - self.pctsp.cost[city_l][0]
            - self.pctsp.penal[city_add]
            + self.pctsp.cost[city_l][city_add]
            + self.pctsp.cost[city_add][0])
        
        self.size += 1
        self.prize += self.pctsp.prize[city_add]

    def remove_city(self, index):
        city_rem = self._route[index]
        city_rem_prev = self._route[index-1]
        city_rem_next = self._route[(index+1)%self.size]

        self.quality = (self.quality
            - self.pctsp.cost[city_rem_prev][city_rem] - self.pctsp.cost[city_rem][city_rem_next]
            + self.pctsp.penal[city_rem]
            + self.pctsp.cost[city_rem_prev][city_rem_next])
        self.prize -= self.pctsp.prize[city_rem]

        del self._route[index]        
        self._route.append(city_rem)

        self.size -= 1

    def remove_cities(self, quant):
        for i in range(self.size-quant,self.size):
            city_rem = self._route[i]
            city_rem_prev = self._route[i-1]

            self.quality = (self.quality 
                - self.pctsp.cost[city_rem_prev][city_rem]
                + self.pctsp.penal[city_rem])
            self.prize -= self.pctsp.prize[city_rem]

        city_rem = self._route[self.size-1]
        city_l = self._route[self.size-quant-1]
        self.quality = (self.quality - self.pctsp.cost[city_rem][0]
            + self.pctsp.cost[city_l][0])

        self.size -= quant

    def print_route(self):
        print(self._route)

    @property
    def route(self):
        return self._route

    @route.setter
    def route(self, r):
        self._route = r
        self.compute()

In [7]:
"""
ilocal_search module - Implements Iterate Local Search algorithm.

"""

def ils(s):
    h = s.copy()
    best = s.copy()
    times = random.sample(range(1000, 2000), 10)    

    while len(times) > 0:
        time = times.pop()
        t = 0
        s_tabu = s.copy()
        while t < time:
            r = tweak(s_tabu.copy())
            if r.quality < s_tabu.quality:
                s_tabu = r

                if s_tabu.is_valid():
                    s = s_tabu
            t += 1

        if s.quality < best.quality and s.is_valid():
            best = s
        
        h = newHomeBase(h, s)
        s = perturb(h)
    
    return best

def tweak(solution):
    s = solution

    s_1 = m1(solution.copy())
    s_2 = m2(solution.copy())
    
    if (s_1 and s_1.quality < solution.quality 
        and (not s_2 or s_1.quality < s_2.quality)
        ):#and s_1.is_valid()):
        s = s_1
    elif (s_2 and s_2.quality < solution.quality
        and (not s_1 or s_2.quality < s_1.quality)
        ):#and s_2.is_valid()):
        s = s_2
    else:
        s_3 = m3(solution.copy())
        if (s_3 and s_3.quality < solution.quality
            ):#and s_3.is_valid()):
            s = s_3

    return s

def newHomeBase(h, s):
    if s.quality <= h.quality:
        return s
    else:
        return h

def perturb(solution):
    s = solution.copy()
    if s.size > 5:
        quant = int(s.size/5)
        s.remove_cities(quant=quant)

    return s

def m1(solution):
    size = solution.size
    length = len(solution.route)

    if size > 1 and size < length:
        i = random.randrange(1, size)
        j = random.randrange(size, length)
        solution.swap(i, j)
   
    return solution

def m2(solution):
    if solution.size > 1:
        i = random.randrange(1, solution.size)
        solution.remove_city(index=i)

    return solution

def m3(solution):
    if solution.size < len(solution.route):
        solution.add_city()

    return solution

In [8]:
"""
genius module - Implements GENIUS, an algorithm for generation of a solution.

"""

def genius(pctsp):
    s = solution.random(pctsp, size=3)
    s = geni(pstsp, s)
    s = us(pctsp, s)

    return s

def geni(pctsp, s):
    return

def us(pctsp, s):
    return

In [9]:
"""
geni module - Auxiliary functions to the GENI method.

"""

def geni(v, s, max_i):
    quality_1 = 0
    quality_2 = 0

    s_star = Solution()
    s_start.quality = sys.maxint

    for i in range(1, max_i):
        quality_1 = quality_after_insertion_1(v, i, )
        quality_2 = quality_after_insertion_2()

        if quality_1 < quality_2 and quality_1 < s_star.quality:
            s_star = insertion_1(s)
        elif quality_2 < quality_1 and quality_2 < s_star.quality:
            s_star = insertion_2(s)

    return s_star

In [12]:
"""
application module - Main module that solves the Prize Collecting Travelling Salesman Problem

"""

# INPUT_INSTANCE_FILE = resource_filename('pctsp', 'data/problem_20_100_100_1000.pctsp')
INPUT_INSTANCE_FILE = 'dataset/pctsp/problem_20_100_100_1000.pctsp'

def main():
    """Main function, that solves the PCTSP.

    """
    pctsp = Pctsp()
    pctsp.load(INPUT_INSTANCE_FILE)
    #pctsp.prize = np.array([0, 4, 8, 3])
    #pctsp.penal = np.array([1000, 7, 11, 17])
    #pctsp.cost = np.array([[0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 0, 1], [1, 1, 1, 0]])
    print(pctsp.type)

    size = int(len(pctsp.prize)*0.7)

    s = solution_random(pctsp, size=size)
    print(s.route)
    print(s.size)
    print(s.quality)
    print(s.is_valid())

    print("\n")

    # s = genius(pctsp)
    # print(s.route)
    # print(s.quality)

    s = ils(s)
    print(s.route)
    print(s.size)
    print(s.quality)
    print(s.is_valid())

In [13]:
main()

A
[0, 10, 15, 4, 1, 16, 17, 5, 13, 3, 11, 18, 9, 2, 14, 7, 12, 19, 6, 8]
14
5277
True


[0, 10, 15, 4, 16, 17, 5, 13, 3, 11, 18, 2, 14, 7, 12, 19, 6, 8, 1, 9]
12
4764
True
